# Text Classifier

In [1]:
pip install transformers evaluate accelerate datasets scikit-learn torch tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.4 MB/s eta 0:00:00
  Using cached pyarrow-16.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.0 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (31 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 8.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=02b4f6b42155aa02ba350fd5d24470db0ca086e2f5fb840166083234aee29155
  Stored in directory: /Users/jameskim/Library/Caches/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
train_df = pd.read_csv('ALLtraincompiled.csv')
test_df = pd.read_csv('ALLtestcompiled.csv')

train_df = train_df[['Question','Type']]
test_df = test_df[['Question','Type']]

# Split the dataset into training and test sets
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define the label mapping
label_mapping = {'counting_and_probability': 0, 'intermediate_algebra': 1, 'number_theory': 2, 'precalculus':3, 
                'prealgebra':4, 'geometry':5, 'algebra':6}
train_df['Type'] = train_df['Type'].map(label_mapping)
test_df['Type'] = test_df['Type'].map(label_mapping)

In [10]:
train_df['Type'].value_counts()

Type
6    1744
1    1295
4    1205
5     870
2     869
0     771
3     746
Name: count, dtype: int64

In [3]:
test_df['Type'].value_counts()

Type
6    1187
1     903
4     871
3     546
2     540
5     479
0     474
Name: count, dtype: int64

In [48]:
import re

def remove_commas_from_latex_numbers(text):
    # Regular expression to match numbers with commas
    pattern = r'(\$[1-9]\d{0,2}(,\!?\d{3})*(\.\d+)?\$)'

    # Function to remove commas from matched numbers
    def remove_commas(match):
        return match.group().replace(',', '').replace(r'\!', '')

    # Replace commas in matched numbers
    result = re.sub(pattern, remove_commas, text)
    return result

# Example input string
input_text = r"""
Cedric has deposited $\$12,\!000$ into an account that pays $5\%$ interest compounded annually.

Daniel has deposited $\$12,\!000$ into an account that pays $7\%$ simple annual interest.

In $15$ years Cedric and Daniel compare their respective balances. To the nearest dollar, what is the positive difference between their balances?
"""

# Apply the function to the input string
output_text = remove_commas_from_latex_numbers(input_text)

# Print the output string
print(output_text)



Cedric has deposited $\$12,\!000$ into an account that pays $5\%$ interest compounded annually.

Daniel has deposited $\$12,\!000$ into an account that pays $7\%$ simple annual interest.

In $15$ years Cedric and Daniel compare their respective balances. To the nearest dollar, what is the positive difference between their balances?



In [42]:
import re
from pylatexenc.latex2text import LatexNodes2Text


def remove_comma_from_latex(latex_list):
    # Regular expression to match numbers with commas
    pattern = r'\$([1-9][0-9]{3,}|[1-9][0-9]{5,})'  
    
    # Function to remove commas from matched numbers
    def remove_comma(match):
        return match.group().replace(',', '')
    
    # Replace commas in numbers >= 1000 for each element in the list
    modified_latex_list = []
    for latex in latex_list:
        modified_latex = re.sub(pattern, remove_comma, latex)
        modified_latex_list.append(modified_latex)
        
    return modified_latex_list
# Test the function with an example
# latex_value = r"$12,\!000$"


# Function to extract LaTeX phrases
def extract_latex(text):
    return re.findall(r'\$(.*?)\$', text)

# Function to convert LaTeX to plain text
def convert_latex_to_text(latex_phrases):
    converter = LatexNodes2Text()
    return [converter.latex_to_text(phrase) for phrase in latex_phrases]

def replace_latex_with_text(question, latex_phrases, text_phrases):
    for latex, text in zip(latex_phrases, text_phrases):
        question = question.replace(f"${latex}$", text)
    return question




# Extract LaTeX phrases and convert them
# train_df['Question'] = train_df['Question'].apply(remove_comma_from_latex)
train_df['Latex_Phrases'] = train_df['Question'].apply(extract_latex)
train_df['Math_Text'] = train_df['Latex_Phrases'].apply(convert_latex_to_text)
train_df['New_Question'] = train_df.apply(lambda row: replace_latex_with_text(row['Question'], row['Latex_Phrases'], row['Math_Text']), axis=1)


# # Print the DataFrame
print(train_df[['Question','Latex_Phrases','Math_Text','New_Question']])
train_df.to_csv('questions_with_plain_text.csv', index=False)

# train_df['Question']

                                               Question  \
0     A board game spinner is divided into three par...   
1     My school's math club has 6 boys and 8 girls. ...   
2     How many 4-letter words with at least one cons...   
3     Melinda will roll two standard six-sided dice ...   
4     Let $p$ be the probability that, in the proces...   
...                                                 ...   
7495  If $\displaystyle\frac{q}{r} = 9$, $\displayst...   
7496              Factor the expression $x(x+2)+(x+2)$.   
7497  Cedric has deposited $\$12,\!000$ into an acco...   
7498  Rural School USA has 105 students enrolled. Th...   
7499  Jo adds up all the positive integers from 1 to...   

                                          Latex_Phrases  \
0         [A, B, C, A, \frac{1}{3}, B, \frac{5}{12}, C]   
1                                                    []   
2                              [A, B, C, D, E, B, C, D]   
3                                                    []

In [12]:
train_df['New_Question'].iloc[8]

'The number of increasing sequences of positive integers a_1 ≤a_2 ≤a_3 ≤⋯≤a_10 ≤2007 such that a_i-i is even for 1≤i ≤10 can be expressed as m n for some positive integers m > n. Compute the remainder when m is divided by 1000.\n'

https://github.com/purdue-tlt/latex2sympy/tree/master

Use this in terminal to download

pip install wheel
pip install "cython<3.0.0" && pip install --no-build-isolation pyyaml==6.0 


In [1]:
pip install latex2sympy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 5.1 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.7.2-py3-none-any.whl size=140931 sha256=929bede0eda7eb678ffa16b4431046a1067e1fe97d7dae34f879106297706035
  Stored in directory: /Users/jameskim/Library/Caches/pip/wheels/24/09/d5/906109a0b239813cee4d939fa6ccee45f63a19e3f774405e34
Successfully built antlr4-python3-runtime
Note: you may need to restart the kernel to use updated packages.


In [49]:
latexx= '$\$12,\!000$'
latex2sympy(latexx)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/var/folders/bs/zggx74yj4d721rnp57yp6h5r0000gn/T/ipykernel_52452/721610512.py:1: SyntaxWarning: invalid escape sequence '\$'
  latexx= '$\$12,\!000$'


Exception: I don't understand this
 \ 12,\!000 
~^

In [6]:
from latex2sympy2 import latex2sympy

train_df['sympy_column'] = train_df['latex_column'].apply(convert_to_sympy)
print(x)
# => "diff(x**(2), x)"

binomial(m, n)


<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
/var/folders/bs/zggx74yj4d721rnp57yp6h5r0000gn/T/ipykernel_52452/3262760491.py:3: SyntaxWarning: invalid escape sequence '\c'
  x = latex2sympy("${m \choose n}$")


In [33]:
from datasets import Dataset
from transformers import BertTokenizer

# Convert DataFrames to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', add_special_tokens=True)


latex_pattern = r'\$([^$]*)\$'

# Tokenization function with LaTeX support
def tokenize_function(examples):
    # Tokenize each problem while preserving LaTeX
    problem_text = examples['Question']
    # Find LaTeX expressions and tokenize them separately
    matches = re.finditer(latex_pattern, problem_text)
    tokenized_problem = []
    last_end = 0
    for match in matches:
        start, end = match.span()
        # Tokenize text before LaTeX expression
        tokenized_problem.extend(tokenizer.tokenize(problem_text[last_end:start]))
        # Tokenize LaTeX expression and add each token separately
        latex_expression = match.group(1)
        tokenized_latex = tokenizer.tokenize(latex_expression)
        tokenized_problem.extend(tokenized_latex)
        last_end = end
    # Tokenize the remaining text after the last LaTeX expression
    tokenized_problem.extend(tokenizer.tokenize(problem_text[last_end:]))
    return tokenizer.pad({'input_ids': tokenized_problem}, return_tensors='pt')

# Tokenization function
# Tokenization function with debugging
# def tokenize_function(examples):
#     tokenized_inputs = tokenizer(examples['Question'], padding='max_length', truncation=True)
#     # print("Example:", examples['Question'])
#     # print("Tokenized inputs:", tokenized_inputs)
#     return tokenized_inputs


# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


# Set format for PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Type'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Type'])


Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

TypeError: expected string or bytes-like object, got 'list'

In [14]:
pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.6 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
import torch
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# Load BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Apply LoRA to the BERT model
config = LoraConfig(
    r=8,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor for the LoRA updates
    target_modules=["query", "key", "value"],  # Apply LoRA to these layers
    lora_dropout=0.1,  # Dropout rate
)

model = get_peft_model(model, config)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/homebrew/Cellar/jupyterlab/4.2.0/libexec/lib/python3.12/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


IndexError: Invalid key: 7278 is out of bounds for size 0